[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meeslindhout/MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start/blob/main/1.%20G%20Collab%20-%20Training_an_Agent_for_Retailrocket.ipynb)


Warning! <font color="red">Google collab has maximum session lengths of 1 hour, after 1 hour of running a jupyter notebook it will automatically shut down. This means that if you are training a model, you will lose all progress after 1 hour when you decide to train multiple agents in one session. Therefore, it is recommended to train one agent at a time and save the model weights immediately after training!</font>  

On average it takes about 10 minutes to train an agent for 3000 episodes.

If you would like to immediately use a a pre-trained agent, you can download the pre-trained agent from the following link: [Pre-trained agents](https://icthva-my.sharepoint.com/:f:/g/personal/mees_lindhout_hva_nl/EjZqaU7dDDhNlYVhK2PJT18BKCpxamzrmgyMT9tIHuk-kQ?e=JgkLKM)

## Accessing the datasets from Google Drive & cloning the Github repo

In [ ]:
# connect to google drive to retrieve our preprocessed datasets.
from google.colab import drive
drive.mount('/content/drive')
import os

In [ ]:
# the processed datasets folder needs to be moved to the github data folder
!ls "drive/MyDrive/Thesis research project"

In [ ]:
# Clone the github repo that contains the code of the trainig agent
!git clone https://github.com/meeslindhout/MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start.git
# you can ignore the lfs error.

In [ ]:
# At the moment our data folder does not contain the preprocessed data that is currently in our google drive. We need to copy it from google drive to the github repo
!ls MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start/data

In [ ]:
# Copy the processed data folder to the data folder in the github repo
!cp -r "/content/drive/MyDrive/Thesis research project/processed datasets" MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start/data

In [ ]:
# Now, our data folder does contain the preprocessed data folder and we are ready to go
!ls MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start/data

In [ ]:
# set the working directory to be our github repo
os.chdir('MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start')

In [ ]:
# install required packages
!pip install wandb -qU

In [ ]:
# load all required libraries
import pandas as pd
from src.recsys_rl import rl_recommender
import wandb
from google.colab import runtime

In [ ]:
# login to wandb to monitor agent training
wandb.login()

## Load the dataset

In [6]:
train = pd.read_csv(r'data/premade datasets/retailrocket/events_train_full.txt',
                    sep='\t')
display(train.head())
display(train.shape)

,Time,UserId,Type,ItemId,SessionId
0,1442004589,0,view,285930,1
1,1442004759,0,view,357564,1
2,1442004917,0,view,67045,1
3,1438969904,2,view,325215,3
4,1438970013,2,view,325215,3


(1184988, 5)

In [7]:
# how many sessions are in the training set?
train['SessionId'].nunique()

321871

In [8]:
# get unique values from type
train['Type'].unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

In [11]:
# replace view with 0, and purchase with 1
train['Type'] = train['Type'].replace({'view': 0, 'addtocart':1, 'transaction': 2})

In [ ]:
# initialize the model / agent
model = rl_recommender(
    n_history=5,
    reward_dict={0:3, 1:8, 3:10},
    event_key='Type',
    mode='training',
    num_episodes=3000,
    batch_size=64,
    target_update_freq=1000,
    memory=100_000,
    learning_rate=0.0003,
    gamma=0.99,
    dataset_name='retailrocket',
    custom_wandb_note=''
    )

In [ ]:
model.fit(train=train)

In [ ]:
# copy all trained agents to google drive
!cp -r "/content/MDDB-Master-thesis-handin---Offline-Batch-Reinforcement-Learning-to-mitigate-Cold-start/trained agents/" "/content/drive/MyDrive/Thesis research project/"

In [ ]:
# unassign the runtime from google collab. 
# This is to prevent the runtime from being disconnected after 12 hours.
runtime.unassign()